### Anaconda Terminal에 들어가 Visdom을 먼저 실행시키자!
`python -m visdom.server`

## Import Visdom and other libraries

In [1]:
import torch
import torch.nn as nn

import torchvision
import torchvision.datasets as dsets

import visdom
vis = visdom.Visdom()

Setting up a new session...


## Text

In [2]:
vis.text("Hello, world!",env="main")

'window_38ef2d2230c7d8'

env는 없어도 되지만 한번에 모든 창을 관리할 때 편리!

## Image

In [3]:
a=torch.randn(3,200,200)
vis.image(a)

'window_38ef2d2237c04c'

## Images

In [4]:
vis.images(torch.Tensor(3,3,28,28))

'window_38ef2d22394576'

## Example (using MNIST and CIFAR10)

In [5]:
# 시간이 좀 걸립니다.
MNIST = dsets.MNIST(root="./MNIST_data",train = True,transform=torchvision.transforms.ToTensor(), download=True)
cifar10 = dsets.CIFAR10(root="./cifar10",train = True, transform=torchvision.transforms.ToTensor(),download=True)

Files already downloaded and verified


#### CIFAR10

In [6]:
data = cifar10.__getitem__(0)
print(data[0].shape)
vis.images(data[0],env="main")

torch.Size([3, 32, 32])


'window_38ef2d22cc464c'

#### MNIST

In [7]:
data = MNIST.__getitem__(0)
print(data[0].shape)
vis.images(data[0],env="main")

torch.Size([1, 28, 28])


'window_38ef2d22ce8f64'

#### Check dataset

In [8]:
data_loader = torch.utils.data.DataLoader(dataset = MNIST,
                                          batch_size = 32,
                                          shuffle = False)

In [9]:
# data_loader에 담겨있는 data들 한번에 보기
for num, value in enumerate(data_loader):
    value = value[0]
    print(value.shape)
    vis.images(value)
    
    break

torch.Size([32, 1, 28, 28])


In [10]:
# main에 있는 모든 창들 다 닫기
vis.close(env="main")

''

## Line Plot

In [11]:
# x축을 따로 선언하지 않으면 0~1사이의 값으로 설정이 된다.
Y_data = torch.randn(5)
plt = vis.line (Y=Y_data)

In [12]:
X_data = torch.Tensor([1,2,3,4,5])
plt = vis.line(Y=Y_data, X=X_data)

### Line update

loss 값 같은 것은 하나씩 나오게 되는데 그 값을 이용하여 plot을 update하는 방법

In [13]:
Y_append = torch.randn(1)
X_append= torch.Tensor([6])

vis.line(Y=Y_append, X=X_append, win=plt, update='append')
# y축 update, x축 update, window 이름, update 방식

'window_38ef2d22d98474'

### Multiple Line on single windows

In [14]:
num = torch.Tensor(list(range(0,10))) # num.shape(10,)
num = num.view(-1,1)                  # num.shape(10,1)
num = torch.cat((num,num),dim=1)      # num.shape(10,2)
print(num)
plt = vis.line(Y=torch.randn(10,2), X = num)

tensor([[0., 0.],
        [1., 1.],
        [2., 2.],
        [3., 3.],
        [4., 4.],
        [5., 5.],
        [6., 6.],
        [7., 7.],
        [8., 8.],
        [9., 9.]])


### Line info

In [15]:
# y_data, x_data, opts = dict(그래프의 이름, 축의 이름 붙이기)
plt = vis.line(Y=Y_data, X=X_data, opts = dict(title='Test', showlegend=True))

In [16]:
# y_data, x_data, opts = dict(그래프의 이름, 축의 이름 자체를 붙이기)
plt = vis.line(Y=Y_data, X=X_data, opts = dict(title='Test', legend = ['1번'],showlegend=True))

In [17]:
# 2개의 그래프에 대해서 그래프의 이름, 축의 이름 붙이기
plt = vis.line(Y=torch.randn(10,2), X = num, opts=dict(title='Test', legend=['1번','2번'],showlegend=True))

## Make function for update line

loss value가 update가 어떻게 되는지 알아볼 때 사용

In [18]:
def loss_tracker(loss_plot, loss_value, num):
    vis.line(X = num,
             Y = loss_value,
             win = loss_plot,
             update = 'append')

In [19]:
plt = vis.line(Y=torch.Tensor(1).zero_())

for i in range(500):
    loss = torch.randn(1) + i
    loss_tracker(plt, loss, torch.Tensor([i]))

## close the window

In [20]:
vis.close(env="main")

''